In [1]:
import numpy as np
import pandas as pd
import awkward as ak
import tensorflow as tf
import matplotlib.pyplot as plt
import topcoffea.modules.utils as utils
import pickle
import gzip

2025-03-27 11:14:08.897721: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
fSMEFT = "dctr_SMEFT.pkl.gz"
fpowheg = "dctr_powheg.pkl.gz"
inputs_smeft= pickle.load(gzip.open(fSMEFT)).get()
inputs_powheg = pickle.load(gzip.open(fpowheg)).get()

In [15]:
inputs_smeft

,weights,avg_top_pt,mtt,top1pt,top1eta,top1phi,top1mass,top2pt,top2eta,top2phi,top2mass
0,1,121.091652,405.697418,201.504684,0.207336,0.275452,171.250,77.005112,1.421631,1.445068,173.875
1,1,174.104477,457.441895,61.577915,0.751465,-2.101074,170.250,287.320770,1.357422,-2.266113,183.875
2,1,26.842417,585.330017,234.615784,-0.141693,2.534180,180.000,208.530914,0.548828,-0.819946,173.250
3,1,24.866749,592.267273,128.457825,-1.387451,-0.256897,172.625,129.100266,1.089600,2.496094,173.750
4,1,20.160759,526.560059,198.016953,-0.920044,1.007080,172.500,199.054947,-0.725220,-2.337891,173.000
...,...,...,...,...,...,...,...,...,...,...,...
1958484,1,39.687988,937.977356,370.888275,0.401489,-0.174347,167.000,345.391449,-0.917847,2.756836,171.875
1958485,1,31.448212,528.829163,194.120712,1.029785,2.129395,172.625,133.585770,2.235840,-0.906128,173.750
1958486,1,12.184126,369.474091,48.234180,-0.023140,-1.714600,171.375,58.687305,1.197021,1.010254,172.375
1958487,1,36.724125,368.968658,88.141495,2.988770,2.972168,171.875,38.239594,3.301758,-1.134766,171.125


In [16]:
inputs_powheg

,weights,avg_top_pt,mtt,top1pt,top1eta,top1phi,top1mass,top2pt,top2eta,top2phi,top2mass
0,1,44.449451,501.201172,207.00000,-1.105469,-1.070312,172.058716,134.50,-1.941406,1.761719,173.449829
1,1,56.248192,449.773193,116.25000,-0.562500,0.962891,172.957275,47.75,2.070312,2.820312,173.172729
2,1,136.914505,484.864258,218.50000,1.144531,0.005753,172.730225,64.50,-0.843750,0.617188,172.158691
3,1,6.896573,734.594116,75.50000,-2.703125,-0.081787,172.463989,78.00,1.449219,2.882812,173.686523
4,1,49.976830,354.261871,35.00000,-0.188477,-0.548828,170.849976,100.00,-0.364258,1.199219,164.613281
...,...,...,...,...,...,...,...,...,...,...,...
21155760,1,26.926012,700.370361,102.00000,-2.054688,0.984375,171.860352,65.75,1.902344,-2.648438,174.025513
21155761,1,60.619133,549.063538,20.12500,0.542969,0.876953,169.948486,101.50,2.687500,0.662109,172.781616
21155762,1,73.218552,578.909119,168.50000,-0.591797,0.664062,172.036865,311.00,-0.388672,-2.625000,172.891113
21155763,1,21.913364,396.134674,101.25000,0.589844,2.375000,170.066528,67.50,1.722656,-0.426758,172.989624


In [3]:
print(inputs_smeft['weights'].shape)
print(inputs_powheg['weights'].shape)

(1958489,)
(21155765,)


In [4]:
# z_0 = inputs_smeft.to_numpy()
# z_1 = inputs_powheg.dropna().to_numpy()

z_0 = inputs_smeft
z_1 = inputs_powheg.iloc[:1958489]

w_0 = inputs_smeft['weights']
w_1 = z_1['weights']

y_0 = np.ones_like(z_0['weights'])
y_1 = np.zeros_like(z_1['weights'])

In [5]:
z = np.concatenate([z_0, z_1], axis=0).astype(np.float32)
w = np.concatenate([w_0, w_1], axis=0).astype(np.float32)
y = np.concatenate([y_0, y_1], axis=0).astype(np.float32)

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((z, y, w)).shuffle(len(z)).batch(128)

2025-03-27 11:28:49.663773: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 64. Tune using inter_op_parallelism_threads for best performance.


In [7]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(z)

In [8]:
# Define the neural network model
def create_model(input_dim):
    model = tf.keras.Sequential([
        normalizer, 
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(300, activation='relu'),
        tf.keras.layers.Dense(300, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

In [9]:
# Define custom loss function
def custom_loss(y_true, y_pred, weights):
    term_1 = tf.reduce_sum(weights * y_true * -tf.math.log(y_pred + 1e-8))
    term_0 = tf.reduce_sum(weights * (1 - y_true) * -tf.math.log(1 - y_pred + 1e-8))
    return term_1 + term_0

In [10]:
# Training step
@tf.function
def train_step(x, y, weights):
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = custom_loss(y, predictions, weights)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [11]:
# Compute r(z)
def compute_r(z):
    f_z = model.predict(z)
    return f_z / (1 - f_z + 1e-8)

In [12]:
# Initialize the model
input_dim = z.shape[1]
model = create_model(input_dim)

In [13]:
# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [14]:
# Training loop
epochs = 20
for epoch in range(epochs):
    epoch_loss = 0
    for x_batch, y_batch, w_batch in dataset:
        loss = train_step(x_batch, tf.expand_dims(y_batch, axis=-1), w_batch)
        epoch_loss += loss.numpy()
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

2025-03-27 11:29:46.431881: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [3916978]
	 [[{{node Placeholder/_1}}]]
2025-03-27 11:29:46.432119: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [3916978]
	 [[{{node Placeholder/_2}}]]


Epoch 1, Loss: 306541433.8429
Epoch 2, Loss: 305127244.6965
Epoch 3, Loss: 304782257.0225
Epoch 4, Loss: 304589684.3286
Epoch 5, Loss: 304484628.5034
Epoch 6, Loss: 304403291.1860
Epoch 7, Loss: 304330110.7238
Epoch 8, Loss: 304289925.2203
Epoch 9, Loss: 304229446.3636
Epoch 10, Loss: 304175500.9106
Epoch 11, Loss: 304152936.3239
Epoch 12, Loss: 304131378.3053
Epoch 13, Loss: 304107673.5288



KeyboardInterrupt



In [ ]:
r_z_0 = compute_r(inputs_smeft)
r_z_1 = compute_r(inputs_powheg)